## Importing Libraries

In [2]:
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from prettytable import PrettyTable
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import STOPWORDS, WordCloud
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re, string, unicodedata
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import warnings
warnings.filterwarnings('ignore')

## Loading the data

In [4]:
%%time
df = pd.read_csv("Dataset\IMDB-Dataset.csv", encoding='latin-1')
df.head()

CPU times: total: 6.22 s
Wall time: 8.87 s


,Ratings,Reviews,Movies,Resenhas
0,1.0,*Disclaimer: I only watched this movie as a co...,Disaster Movie,* IsenÃ§Ã£o de responsabilidade: eu sÃ³ assist...
1,1.0,I am writing this in hopes that this gets put ...,Disaster Movie,Estou escrevendo isso na esperanÃ§a de que iss...
2,1.0,"Really, I could write a scathing review of thi...",Disaster Movie,"Realmente, eu poderia escrever uma crÃ­tica co..."
3,1.0,If you saw the other previous spoof movies by ...,Disaster Movie,Se vocÃª viu os outros filmes falsificados ant...
4,1.0,This movie I saw a day early for free and I st...,Disaster Movie,Este filme eu vi um dia cedo de graÃ§a e ainda...


## Stopwords

In [6]:
#Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would","shall","could","might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words=set(stop_words)
print(stop_words)

{'hasn', "mightn't", 'where', 'does', 'a', 'shan', 've', 'am', 'most', 'couldn', 'so', "should've", "it's", "needn't", 'at', 'you', 'he', 'won', 'itself', 'here', 'o', 'yourself', 'm', 'against', 't', 'haven', 'more', 'will', 'up', 'there', 'we', 'few', 'ourselves', 'of', 'all', 'before', 'has', 'being', 'doesn', 'again', 'or', "weren't", 'i', 'by', 'over', 'weren', 'yourselves', 'these', 'him', 'out', 'might', 'then', 'my', 'ours', 'mustn', 'once', 'with', 'his', 'from', 'theirs', 'whom', 'is', 'for', 'what', "wasn't", 'only', 'after', 'down', 'own', 'be', "mustn't", 'some', 'should', 'don', 'isn', 'nor', 'but', 'while', 'they', 'hers', "that'll", "hadn't", "hasn't", 'wasn', 'having', 'because', 'no', 'me', 'himself', 'mightn', 'can', "you're", 'those', 'through', 'ain', "haven't", 'to', 'off', 'such', 'when', 'wouldn', 'are', 'on', "shan't", 'their', 'who', 'yours', 'below', 'ma', "you've", 'them', 'any', 'until', "she's", 'above', 'its', 'needn', 'too', "won't", "you'll", 'have', 't

In [7]:
#Removing special character
def remove_special_character(content):
    return re.sub('\W+',' ', content )

# Removing URL's
def remove_url(content):
    return re.sub(r'http\S+', '', content)

#Removing the stopwords
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

# Expansion of english contractions
def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"weren\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)

    return content

#Data preprocessing
def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)
    content = remove_stopwords(content)   
    
    return content

In [8]:
pd.options.display.max_colwidth = 1000

df['Cleaned_Reviews']=df['Reviews'].apply(data_cleaning)
df.head()

,Ratings,Reviews,Movies,Resenhas,Cleaned_Reviews
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* IsenÃ§Ã£o de responsabilidade: eu sÃ³ assisti esse filme como um acordo condicional. E eu vejo filmes de graÃ§a. Eu nÃ£o seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crÃ­tica mais curta de todos os tempos. NÃ£o vÃª este filme. Ã de longe o filme mais estÃºpido, lamenta, preguiÃ§oso e inacreditavelmente UNFUNNY que eu jÃ¡ vi. Ã um desastre total. Mas como o meu Ã³dio por este filme e por outros, se estende muito alÃ©m de uma exibiÃ§Ã£o, acho que vou continuar um pouco. NÃ£o conheÃ§o nenhuma das pessoas do filme alÃ©m de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso nÃ£o importa. Eles sÃ£o todos horrÃ­veis, embora eu ache que esse seja o ponto. A ediÃ§Ã£o Ã© horrÃ­vel e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrÃ­vel do que eu pensava. Agora eu sei que esses filmes nÃ£o devem ser sÃ©rios, mas vamos lÃ¡, Ã© o cinema 101 que se alguÃ©m f...",disclaimer watched movie conditional agreement see films free not caught dead giving hard earned money idiots well explain depth film write shortest review ever not see movie far stupidest lamest lazy unbelievably unfunny movie ever seen total disaster since hatred movie others like extends far beyond one viewing think go bit not know people movie besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point editing flat horrible possibly blatant continuity errors make crapfast even crappier thought know films not supposed serious come film making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia films get away give disaster movie pass jokes thoughtless mindless physical gags obviously take popular movies last year late well including best picture nominees know saddest thing stupid movies not care much money make many cameos sorry ass excuses films taking away jobs actors writers directors truly deserv...
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these 

## Feature Engineering

In [10]:
# Maping Labels to ratings -- 1(positive) 0(negative) 2(neutral)
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >=7 else ('0' if x<=4 else '2'))

# Removing the neutral ratings
df = df[df['Label']<'2']
data = df[['Cleaned_Reviews','Label']]
data['Label'].value_counts()

Label
0    60000
1    60000
Name: count, dtype: int64

## Lemmatization

In [12]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews) ]

# Vectorization with CountVectorizer and TfidfVectorizer with Unigram

In [14]:
train, test = train_test_split(data, test_size=0.3, random_state=31, shuffle=True)

countvec = CountVectorizer(analyzer='word', tokenizer=LemmaTokenizer(), ngram_range=(1,1), min_df=10, max_features=500)
tfidfvec = TfidfVectorizer(analyzer='word', tokenizer=LemmaTokenizer(), ngram_range=(1,1), min_df=10, max_features=500)

X_train_count = countvec.fit_transform(train['Cleaned_Reviews']).toarray()
X_test_count = countvec.transform(test['Cleaned_Reviews']).toarray()

X_train_tfidf = tfidfvec.fit_transform(train['Cleaned_Reviews']).toarray()
X_test_tfidf = tfidfvec.transform(test['Cleaned_Reviews']).toarray()

y_train = train['Label']
y_test = test['Label']

## Feature importance with Logistic Regression and CountVectorizer with Unigram

In [16]:
lgr =  LogisticRegression()
lgr.fit(X_train_count,y_train)
lgr.score(X_test_count,y_test)
lgr.coef_[0]

important_features = PrettyTable(['Features','Score'])

i = 0
for feature, importance in zip(countvec.get_feature_names_out(), lgr.coef_[0]):
    if i<200:
        important_features.add_row([feature,importance])
        i=+1
print(important_features)

+----------------+-------------------------+
|    Features    |          Score          |
+----------------+-------------------------+
|      able      |   0.11608690065746866   |
|   absolutely   |   0.04753651303243684   |
|      act       |   -0.12566448136625963  |
|     acting     |   -0.1904902502660187   |
|     action     |   0.26403611510296027   |
|     actor      |   -0.1391424517324975   |
|    actress     |   -0.11849461333527206  |
|    actually    |  -0.021630656551140322  |
|      add       |   0.02020415631268698   |
|     adult      |   0.11124553874926574   |
|      age       |   0.10432452390325694   |
|     alien      |   -0.08052124063935574  |
|     almost     |  0.0003014038607170288  |
|     alone      |   0.10174455370226441   |
|     along      |   0.12783502923042298   |
|    already     |   -0.2932164370361751   |
|      also      |   0.09868935851173354   |
|    although    |   0.14741341832067031   |
|     always     |    0.1578898220010056   |
|    amazi

## Feature importance with Logistic Regression and TfidfVectorizer with Unigram

In [18]:
lgr =  LogisticRegression()
lgr.fit(X_train_tfidf,y_train)
lgr.score(X_test_tfidf,y_test)
lgr.coef_[0]

important_features = PrettyTable(['Features','Score'])

i = 0
for feature, importance in zip(tfidfvec.get_feature_names_out(), lgr.coef_[0]):
    if i<200:
        important_features.add_row([feature,importance])
        i=+1
print(important_features)

+----------------+-----------------------+
|    Features    |         Score         |
+----------------+-----------------------+
|      able      |   0.4768111255083373  |
|   absolutely   |   0.4052216965355131  |
|      act       |  -1.1609971066499207  |
|     acting     |  -1.6384106338393774  |
|     action     |   2.763503556919685   |
|     actor      |  -1.4354740407747175  |
|    actress     |  -0.7676249452236021  |
|    actually    |  -0.14490874694316616 |
|      add       | -0.022134191985831803 |
|     adult      |   0.8540606549666568  |
|      age       |   0.9000417345007163  |
|     alien      |  -0.3858401399364395  |
|     almost     |  -0.3463072097706616  |
|     alone      |   0.5001932982378909  |
|     along      |   0.8157427190561398  |
|    already     |  -1.4334361045251394  |
|      also      |   1.1930982518143531  |
|    although    |   0.8242494514675376  |
|     always     |   1.6079467840955963  |
|    amazing     |   4.7469874845316635  |
|    americ

# Vectorization with CountVectorizer and TfidfVectorizer with Bigram

In [20]:
train, test = train_test_split(data, test_size=0.3, random_state=31, shuffle=True)

countvec = CountVectorizer(analyzer='word', tokenizer=LemmaTokenizer(), ngram_range=(2,2), min_df=10, max_features=500)
tfidfvec = TfidfVectorizer(analyzer='word', tokenizer=LemmaTokenizer(), ngram_range=(2,2), min_df=10, max_features=500)

X_train_count = countvec.fit_transform(train['Cleaned_Reviews']).toarray()
X_test_count = countvec.transform(test['Cleaned_Reviews']).toarray()

X_train_tfidf = tfidfvec.fit_transform(train['Cleaned_Reviews']).toarray()
X_test_tfidf = tfidfvec.transform(test['Cleaned_Reviews']).toarray()

y_train = train['Label']
y_test = test['Label']

## Feature importance with Logistic Regression and CountVectorizer with Bigram

In [22]:
lgr =  LogisticRegression()
lgr.fit(X_train_count,y_train)
lgr.score(X_test_count,y_test)
lgr.coef_[0]

important_features = PrettyTable(['Features','Score'])

i = 0
for feature, importance in zip(countvec.get_feature_names_out(), lgr.coef_[0]):
    if i<200:
        important_features.add_row([feature,importance])
        i=+1
print(important_features)

+-----------------------+------------------------+
|        Features       |         Score          |
+-----------------------+------------------------+
|       acting bad      |  -1.2661469338407736   |
|      acting good      |   0.7385399287204168   |
|       acting not      |  -0.44242575783613963  |
|      action film      |  0.41638865865896746   |
|      action flick     |   0.4138197707060946   |
|      action movie     |   0.2876690212830454   |
|     action packed     |   1.3291008119477918   |
|      action scene     |   0.1280350357841698   |
|    action sequence    |   0.1390376518883124   |
|     actor actress     |  -0.24004247763631503  |
|       actor not       |  -0.42408076154425406  |
|      almost every     |  -0.1155942214157885   |
|       along way       |  0.41529925341008495   |
|        also not       |  -0.22540849777918312  |
|      although not     |   0.3474805454523109   |
|     anything else     |  -0.3301782963662211   |
|        b movie        |  -0.0

## Feature importance with Logistic Regression and TfidfVectorizer with Bigram

In [24]:
lgr =  LogisticRegression()
lgr.fit(X_train_tfidf,y_train)
lgr.score(X_test_tfidf,y_test)
lgr.coef_[0]

important_features = PrettyTable(['Features','Score'])

i = 0
for feature, importance in zip(tfidfvec.get_feature_names_out(), lgr.coef_[0]):
    if i<200:
        important_features.add_row([feature,importance])
        i=+1
print(important_features)

+-----------------------+------------------------+
|        Features       |         Score          |
+-----------------------+------------------------+
|       acting bad      |  -2.8706605309994404   |
|      acting good      |   1.688499000230906    |
|       acting not      |  -1.2196254874348016   |
|      action film      |   1.2170777169160938   |
|      action flick     |   1.1335569064467308   |
|      action movie     |   0.9219387783927994   |
|     action packed     |   3.0147563003384628   |
|      action scene     |   0.4288993827105423   |
|    action sequence    |  0.43502409289428984   |
|     actor actress     |  -0.4618308553159592   |
|       actor not       |   -0.959355827087467   |
|      almost every     |  -0.2294158376445737   |
|       along way       |   0.9483398797676488   |
|        also not       |   -0.457661948631332   |
|      although not     |   0.7200641531677208   |
|     anything else     |  -0.7515113731807419   |
|        b movie        |  -0.3

# Vectorization with CountVectorizer and TfidfVectorizer with Trigram

In [26]:
train, test = train_test_split(data, test_size=0.3, random_state=31, shuffle=True)

countvec = CountVectorizer(analyzer='word', tokenizer=LemmaTokenizer(), ngram_range=(3,3), min_df=10, max_features=500)
tfidfvec = TfidfVectorizer(analyzer='word', tokenizer=LemmaTokenizer(), ngram_range=(3,3), min_df=10, max_features=500)

X_train_count = countvec.fit_transform(train['Cleaned_Reviews']).toarray()
X_test_count = countvec.transform(test['Cleaned_Reviews']).toarray()

X_train_tfidf = tfidfvec.fit_transform(train['Cleaned_Reviews']).toarray()
X_test_tfidf = tfidfvec.transform(test['Cleaned_Reviews']).toarray()

y_train = train['Label']
y_test = test['Label']

## Feature importance with Logistic Regression and CountVectorizer with Trigram

In [28]:
lgr =  LogisticRegression()
lgr.fit(X_train_count,y_train)
lgr.score(X_test_count,y_test)
lgr.coef_[0]

important_features = PrettyTable(['Features','Score'])

i = 0
for feature, importance in zip(countvec.get_feature_names_out(), lgr.coef_[0]):
    if i<200:
        important_features.add_row([feature,importance])
        i=+1
print(important_features)

+---------------------------+------------------------+
|          Features         |         Score          |
+---------------------------+------------------------+
|       acting not bad      |  -0.5060912361943815   |
|      acting not good      |   -1.111239244041544   |
|      acting not great     |  -0.5837923153430828   |
|     acting pretty good    |   0.7079720594042856   |
|      action movie not     |   1.0119112762701994   |
|      actually not bad     |  -0.6371299570099249   |
|    actually pretty good   |  0.34238244115525507   |
|    actually quite good    |  -0.2827626441957781   |
|     almost every scene    |  -0.20332779559553044  |
|        b movie not        |  -0.08013449397092094  |
|       bad acting bad      |  -2.4190979611522323   |
|        bad bad bad        |   -1.877347602188494   |
|        bad guy not        |  0.41360818801174476   |
|      bad horror movie     |  -0.9765495017626185   |
|       bad movie not       |  -0.5944782626417605   |
|        b

## Feature importance with Logistic Regression and TfidfVectorizer with Trigram

In [30]:
lgr =  LogisticRegression()
lgr.fit(X_train_tfidf,y_train)
lgr.score(X_test_tfidf,y_test)
lgr.coef_[0]

important_features = PrettyTable(['Features','Score'])

i = 0
for feature, importance in zip(tfidfvec.get_feature_names_out(), lgr.coef_[0]):
    if i<200:
        important_features.add_row([feature,importance])
        i=+1
print(important_features)

+---------------------------+-----------------------+
|          Features         |         Score         |
+---------------------------+-----------------------+
|       acting not bad      |  -0.8820368702598814  |
|      acting not good      |  -1.3206602812644355  |
|      acting not great     |  -0.6514267143323558  |
|     acting pretty good    |   0.8887594723985498  |
|      action movie not     |   1.2934620948142908  |
|      actually not bad     |  -0.7281227763609566  |
|    actually pretty good   |   0.4393691918623795  |
|    actually quite good    |  -0.2826926640847789  |
|     almost every scene    |  -0.2799106492607414  |
|        b movie not        |  -0.2768821039731168  |
|       bad acting bad      |   -3.339097448188472  |
|        bad bad bad        |   -2.198495103242972  |
|        bad guy not        |   0.5825872150315924  |
|      bad horror movie     |  -1.2722618299023976  |
|       bad movie not       |  -1.1581146585335138  |
|        bad not even       